In [1]:
import sys
import os

import pandas as pd
import datetime

import django
import platform

if platform.node() == "srv-mcc-apsis":
    #sys.path.append("/home/galm/software/django/tmv/BasicBrowser/")
    sys.path.append('/home/leey/tmv/BasicBrowser/')
else:
    # local paths
    sys.path.append('/home/leey/Documents/Data/tmv/BasicBrowser/')

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

# import from appended path
import scoping.models as sm
import twitter.models as tm
from tmv_app.models import *

import operator
from django.db.models import Q
from functools import reduce

### checking TwitterSearch objects

In [38]:
plist = [196, 228]

In [41]:
ts = tm.TwitterSearch.objects.all().values_list('string', 'project', 'since', 'project_list')

In [42]:
df_ts = pd.DataFrame(list(ts),columns=['string', 'project', 'since', 'project_list'])
df_ts[df_ts.project_list.isin(plist)]

,string,project,since,project_list
0,Kohlekommission,134.0,2009-11-05 09:31:44.480805+00:00,196.0
1,"""Kohleausstieg""",185.0,2010-04-21 14:43:17.352458+00:00,196.0
2,#Hambibleibt,196.0,2010-01-15 02:37:39.062263+00:00,196.0
3,kohlefrei,196.0,NaT,196.0
4,Klima,196.0,NaT,228.0
5,Erderwärmung,196.0,NaT,228.0
6,globale Erwärmung,196.0,NaT,228.0
7,Treibhauseffekt,196.0,NaT,228.0
8,kohle,134.0,2009-11-05 09:31:44.480805+00:00,228.0


---

### Add TwitterSearch objects

In [5]:
# Add climate baseline terms

terms = ['Klima', 'Erderwärmung', 'globale Erwärmung','Treibhauseffekt']

for term in terms:
    t, created = tm.TwitterSearch.objects.get_or_create(
    string = term,
    project = sm.Project.objects.get(pk=196),
    )
    t.project_list.add(sm.Project.objects.get(pk=196))
    t.save()

In [28]:
# amend project association
terms = ['Klima', 'Erderwärmung', 'globale Erwärmung','Treibhauseffekt']

for term in terms:
    t = tm.TwitterSearch.objects.get(
    string = term
    )
    t.project_list.add(sm.Project.objects.get(pk=228))
    t.project_list.remove(sm.Project.objects.get(pk=196))
    t.save()

In [2]:
# Add coal exit terms 

terms = ['kohlefrei']

for term in terms:
    t, created = tm.TwitterSearch.objects.get_or_create(
        string = term,
        project = sm.Project.objects.get(pk=196)
    )
    t.project_list.add(sm.Project.objects.get(pk=196))
    t.save()

### Add TwitterSearch objects to project_list

In [13]:
plist_terms =  [
    'Kohlekommission',
    '"Kohleausstieg"',
    #'#Kohlekommission',
    #'#Kohleausstieg',
    '#Hambibleibt',
    ]

for term in plist_terms:
    t =  tm.TwitterSearch.objects.get(string=term)
    t.project_list.add(sm.Project.objects.get(pk=196))

In [43]:
t =  tm.TwitterSearch.objects.get(string='kohle')
t.project_list.remove(sm.Project.objects.get(pk=228))

### Add relevant kohle tweets to new project

In [4]:
p1 = sm.Project.objects.get(pk=228)

p1.title = 'Climate baseline tweets'

In [10]:
kt = tm.Status.objects.filter(text__icontains="Kohle")
kt.count()

1079456

In [17]:
# Q lookup must precede any keyword argument
ktf = tm.Status.objects.filter(
    Q(text__icontains = '#hambacherforst') |
    Q(text__icontains = '#fridaysforfuture') |
    Q(text__icontains = '#endegelaende') |
    Q(text__icontains = '#hambacherwald') |
    Q(text__icontains = '#lausitz') |
    Q(text__icontains = '#klimaschutz') |
    Q(text__icontains = '#klimawandel') |
    Q(text__icontains = '#klimakrise') |
    Q(text__icontains = '#klima')
)
#.filter(text__icontains="Kohle")

ktf.count()

1132554

In [18]:
ktff = ktf.filter(
    text__icontains="Kohle"
)
ktff.count()

94008

In [19]:
# add filtered tweets to new project 
# you can't do this - can only add project to twittersearch object! 

### Checking tweets

In [4]:
p = sm.Project.objects.get(pk=196)

In [5]:
end_date = datetime.datetime.strptime('2019-10-01-+0000', '%Y-%m-%d-%z')

In [6]:
# Get tweets related to the project
searches = tm.TwitterSearch.objects.filter(project_list=p)

ttweets = set(tm.Status.objects.filter(
    searches__in=searches,
    text__isnull=False,
    retweeted_status__isnull=True,
    created_at__lte=end_date
).values_list('id',flat=True))

print(len(ttweets))

945086
